Run all this code. The last cell at the bottom will give you all the outputs of the 21 heat index algorithms from Anderson et al. (2013)

In [6]:
import iris
import iris.analysis
import iris.coord_categorisation
import iris.plot
import iris.quickplot
import numpy as np
import math # Use to call euler's number (e)
import os

import basic_info

def save_cube(cube, filename):
    iris.save(cube, filename)
    set_permission(filename)
    print('Saved {}'.format(filename))

os.chdir(basic_info.data_folder)

In [ ]:
tas_filepath = basic_info.tas_filepath
rh_filepath = basic_info.rh_filepath
td_filepath = basic_info.td_filepath
es_filepath = basic_info.es_filepath

tas_cube = iris.load_cube(tas_filepath)
rh_cube = iris.load_cube(rh_filepath)
td_cube = iris.load_cube(td_filepath)
es_cube = iris.load_cube(es_filepath)

In [6]:
### 21 Heat Index Algorithm functions
# Function to calculate Heat Index (HI) using algorithm 1 above
def heat_index_1(location_tas_cube, location_rh_cube):
    """
    Returns the modified heat index (HI) from a tas and h cube.
    
    Parameters
    ----------
    location_tas_cube :
        An instance of `iris.cube.Cube` representing temperature data in ˚C
    location_rh_cube :
        An instance of `iris.cube.Cube` representing humidity data in %
    """
    assert location_tas_cube.units == 'celsius', "Temperature units must be in celsius"
    assert location_rh_cube.units == '%', 'Ensure humidity units are in %'

    tas_cube_fahrenheit = location_tas_cube.copy()
    tas_cube_fahrenheit.convert_units('Fahrenheit')
    tas_data = tas_cube_fahrenheit.data.astype(np.float64)

    humidity_data = location_rh_cube.data.astype(np.float64)

    A = -10.3 + 1.1 * tas_data + 0.047 * humidity_data

    B = (
        -42.379 + 2.04901523 * tas_data + 10.14333127 * humidity_data
        - 0.22475541 * tas_data * humidity_data
        - 6.83783e-3 * tas_data ** 2 - 5.481717e-2 * humidity_data ** 2
        + 1.22874e-3 * tas_data ** 2 * humidity_data
        + 8.5282e-4 * tas_data * humidity_data ** 2
        - 1.99e-6 * tas_data ** 2 * humidity_data ** 2
    )

    hi = np.where(tas_data <= 40, tas_data,
        np.where(A < 79, A,
            np.where((humidity_data < 13) & (80 <= tas_data) & (tas_data <= 112),
                B - ((13 - humidity_data) / 4) * ((17 - (tas_data - 95)) / 17) ** 0.5,
                np.where((humidity_data > 85) & (80 <= tas_data) & (tas_data <= 87),
                    B + 0.02 * (humidity_data - 85) * (87 - tas_data),
                    B
                )
            )
        )
    )

    heat_index_cube = location_tas_cube.copy()
    heat_index_cube.data = (hi-23) * 5/9 # Convert to Celsius
    heat_index_cube.data= np.ma.masked_where(heat_index_cube.data < -10, heat_index_cube.data) # Masking values below -10 that weirdly pop up
    heat_index_cube.long_name = 'Heat Index 1'

    return heat_index_cube

# Function to calculate Heat Index (HI) using algorithm 2 above
def heat_index_2(location_tas_cube, location_td_cube): # Note: specify the cube, not the cube.data
    """Returns the Schoen (2005) heat index from a tas and td cube.
    
    Parameters
    ----------
    location_tas_cube :
        An instance of :class:`iris.cube.Cube`
    location_td_cube :
        An instance of :class:`iris.cube.Cube`
    """   
    assert location_tas_cube.units == 'celsius' and location_td_cube.units == 'celsius', "Ensure cube units are in celsius"

    tas_cube_copy = location_tas_cube.copy()  # Create a copy of the temperature cube
    td_cube_copy = location_td_cube.copy()  # Create a copy of the td cube

    heat_index_2_data = (
        tas_cube_copy.data
        - 1.0799 * (math.e ** (0.03755 * tas_cube_copy.data))
        * (1 - (math.e ** (0.0801 * (td_cube_copy.data - 14))))
    )

    heat_index_cube = tas_cube_copy.copy()  # Create a new cube with the heat index data
    heat_index_cube.data = heat_index_2_data
    heat_index_cube.long_name = 'Heat Index 2'
    
    return heat_index_cube

# Function to calculate Heat Index (HI) using algorithm 3 above
def heat_index_3(location_tas_cube, location_td_cube):
    """
    Returns the Schoen (2005) heat index from a tas and td cube.
    
    Parameters
    ----------
    location_tas_cube :
        An instance of `iris.cube.Cube` representing temperature data in ˚C
    location_td_cube :
        An instance of `iris.cube.Cube` representing dew point temperature data in ˚C
    """
    assert location_tas_cube.units == 'celsius' and location_td_cube.units == 'celsius', "Ensure cube units are in celsius"

    tas_cube_fahrenheit = location_tas_cube.copy()
    tas_cube_fahrenheit.convert_units('Fahrenheit') # Convert temperature cube to Fahrenheit
    tas_data = tas_cube_fahrenheit.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time

    td_cube_fahrenheit = location_td_cube.copy()
    td_cube_fahrenheit.convert_units('Fahrenheit') # Convert dew point temperature cube to Fahrenheit
    dc_data = td_cube_fahrenheit.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time

    heat_index_3_data = (
        tas_data
        - 0.9971 * (math.e ** (0.02086 * tas_data))
        * (1 - (math.e ** (0.0445 * (dc_data - 57.2))))
    )

    # Create a new cube with the heat index data
    heat_index_cube = location_tas_cube.copy()
    heat_index_cube.data = (heat_index_3_data - 32) * 5/9
    heat_index_cube.long_name = 'Heat Index 3'

    return heat_index_cube

# Function to calculate Heat Index (HI) using algorithm 4 above
def heat_index_4(location_tas_cube, location_es_cube):
    """
    Returns the Gaffen and Ross (1999) and Steadman (1984) heat index from a tas and es cube.
    
    Parameters
    ----------
    location_tas_cube :
        An instance of `iris.cube.Cube` representing temperature data in ˚C
    location_es_cube :
        An instance of `iris.cube.Cube` representing water vaour pressure in Kilopascals
    """
    assert location_tas_cube.units == 'celsius', "Ensure temperature units are in celsius"
    assert location_es_cube.units == 'kPa', 'Ensure water vapor pressure units are in Kilopascals'

    tas_cube_copy = location_tas_cube.copy()  # Create a copy of the temperature cube
    es_cube_copy = location_es_cube.copy()  # Create a copy of the es cube

    heat_index_4_data = (
        -1.3 + 0.92 * location_tas_cube.data + 2.2 * location_es_cube.data)

    # Create a new cube with the heat index data
    heat_index_cube = location_tas_cube.copy()
    heat_index_cube.data = heat_index_4_data
    heat_index_cube.long_name = 'Heat Index 4'

    return heat_index_cube

# Function to calculate Heat Index (HI) using algorithm 5 above
def heat_index_5(location_tas_cube, location_rh_cube):
    """
    Returns the El Morjani et al. (2007) and Oka (2011) heat index from a tas and h cube.
    
    Parameters
    ----------
    location_tas_cube :
        An instance of `iris.cube.Cube` representing temperature data in ˚C
    location_rh_cube :
        An instance of `iris.cube.Cube` representing humidity data in %
    """
    assert location_tas_cube.units == 'celsius', "Ensure temperature units are in celsius"
    assert location_rh_cube.units == '%', 'Ensure humidity units are in %'

    # Create a copy of the temperature cube so when converted it doesn't change the original data
    tas_cube_fahrenheit = location_tas_cube.copy()
    tas_cube_fahrenheit.convert_units('Fahrenheit') # Convert temperature cube to Fahrenheit
    
    tas_data = tas_cube_fahrenheit.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time
    humidity_data = location_rh_cube.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time

    heat_index_5_data = (
        -42.379
        + 2.04901523 * tas_data
        + 10.14333127 * humidity_data
        - 0.22475541 * tas_data * humidity_data
        - (6.83783e-3) * tas_data ** 2
        - (5.481717e-2) * humidity_data ** 2
        + (1.22874e-3) * tas_data ** 2 * humidity_data
        + (8.5282e-4) * tas_data * humidity_data ** 2
        - (1.99e-6) * tas_data ** 2 * humidity_data ** 2
    )

    # Apply correction factor
    correction_mask = np.logical_or(tas_data <= 80, humidity_data <= 40)
    heat_index_5_data[correction_mask] = tas_data[correction_mask]

    # Create a new cube with the heat index data
    heat_index_cube = location_tas_cube.copy()
    heat_index_cube.data = (heat_index_5_data -32) * 5/9
    heat_index_cube.long_name = 'Heat Index 5'
    
    return heat_index_cube
    
# Function to calculate Heat Index (HI) using algorithm 6 above
def heat_index_6(location_tas_cube, location_rh_cube):
    """
    Returns the Fandoeva et al. (2009) heat index from a tas and h cube.
    
    Parameters
    ----------
    location_tas_cube :
        An instance of `iris.cube.Cube` representing temperature data in ˚C
    location_rh_cube :
        An instance of `iris.cube.Cube` representing humidity data in %
    """
    assert location_tas_cube.units == 'celsius', "Ensure temperature units are in celsius"
    assert location_rh_cube.units == '%', 'Ensure humidity units are in %'

    tas_cube_fahrenheit = location_tas_cube.copy()
    tas_cube_fahrenheit.convert_units('Fahrenheit') # Convert temperature cube to Fahrenheit
    tas_data = tas_cube_fahrenheit.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time
    
    humidity_data = location_rh_cube.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time

    heat_index_6_data = (
        -42.379
        + 2.04901523 * tas_data
        + 10.14333127 * humidity_data
        - 0.22475541 * tas_data * humidity_data
        - (6.83783e-3) * tas_data ** 2
        - (5.481717e-2) * humidity_data ** 2
        + (1.22874e-3) * tas_data ** 2 * humidity_data
        + (8.5282e-4) * tas_data * humidity_data ** 2
        - (1.99e-6) * tas_data ** 2 * humidity_data ** 2
    )

    # Apply correction factor
    correction_mask = np.logical_or(tas_data < 80, humidity_data < 40)
    heat_index_6_data[correction_mask] = tas_data[correction_mask]

    # Create a new cube with the heat index data
    heat_index_cube = location_tas_cube.copy()
    heat_index_cube.data = (heat_index_6_data -32) * 5/9
    heat_index_cube.long_name = 'Heat Index 6'

    return heat_index_cube

# Function to calculate Heat Index (HI) using algorithm 7 above
def heat_index_7(location_tas_cube, location_rh_cube):
    """
    Returns the Di Cristo et al. (2007) and Rajib et al. (2011) heat index from a tas and h cube.
    
    Parameters
    ----------
    location_tas_cube :
        An instance of `iris.cube.Cube` representing temperature data in ˚C
    location_rh_cube :
        An instance of `iris.cube.Cube` representing humidity data in %
    """
    assert location_tas_cube.units == 'celsius', "Temperature units must be in celsius"
    assert location_rh_cube.units == '%', 'Ensure humidity units are in %'

    tas_cube_fahrenheit = location_tas_cube.copy()
    tas_cube_fahrenheit.convert_units('Fahrenheit') # Convert temperature cube to Fahrenheit
    
    tas_data = tas_cube_fahrenheit.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time
    humidity_data = location_rh_cube.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time

    heat_index_7_data = (
        -42.379
        + 2.04901523 * tas_data
        + 10.14333127 * humidity_data
        - 0.22475541 * tas_data * humidity_data
        - (6.83783e-3) * tas_data ** 2
        - (5.481717e-2) * humidity_data ** 2
        + (1.22874e-3) * tas_data ** 2 * humidity_data
        + (8.5282e-4) * tas_data * humidity_data ** 2
        - (1.99e-6) * tas_data ** 2 * humidity_data ** 2
    )

    # Apply correction factor
    correction_mask = np.logical_or(tas_data <= 78.8, humidity_data <= 39)
    heat_index_7_data[correction_mask] = tas_data[correction_mask]

    # Create a new cube with the heat index data
    heat_index_cube = location_tas_cube.copy()
    heat_index_cube.data = (heat_index_7_data -32) * 5/9
    heat_index_cube.long_name = 'Heat Index 7'

    return heat_index_cube

# Function to calculate Heat Index (HI) using algorithm 8 above
def heat_index_8(location_tas_cube, location_rh_cube):
    """
    Returns the Johnson and Long (2004) heat index from a tas and h cube.
    
    Parameters
    ----------
    location_tas_cube :
        An instance of `iris.cube.Cube` representing temperature data in ˚C
    location_rh_cube :
        An instance of `iris.cube.Cube` representing humidity data in %
    """
    assert location_tas_cube.units == 'celsius', "Temperature units must be in celsius"
    assert location_rh_cube.units == '%', 'Ensure humidity units are in %'

    tas_cube_fahrenheit = location_tas_cube.copy()
    tas_cube_fahrenheit.convert_units('Fahrenheit') # Convert temperature cube to Fahrenheit
    
    tas_data = tas_cube_fahrenheit.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time
    humidity_data = location_rh_cube.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time

    heat_index_8_data = (
        -42.4
        + 2.049 * tas_data
        + 10.14 * humidity_data
        - 0.2248 * tas_data * humidity_data
        - (6.838e-3) * tas_data ** 2
        - (5.482e-2) * humidity_data ** 2
        + (1.229e-3) * tas_data ** 2 * humidity_data
        + (8.528e-4) * tas_data * humidity_data ** 2
        - (1.99e-6) * tas_data ** 2 * humidity_data ** 2
    )

    # Apply correction factor
    correction_mask = tas_data < 79
    heat_index_8_data[correction_mask] = tas_data.data[correction_mask]

    # Create a new cube with the heat index data
    heat_index_cube = location_tas_cube.copy()
    heat_index_cube.data = (heat_index_8_data -32) * 5/9
    heat_index_cube.long_name = 'Heat Index 8'
    
    return heat_index_cube

# Function to calculate Heat Index (HI) using algorithm 9 above
def heat_index_9(location_tas_cube, location_rh_cube):
    """
    Returns the Robinson (2001) heat index from a tas and h cube.
    
    Parameters
    ----------
    location_tas_cube :
        An instance of `iris.cube.Cube` representing temperature data in ˚C
    location_rh_cube :
        An instance of `iris.cube.Cube` representing humidity data in %
    """
    assert location_tas_cube.units == 'celsius', "Temperature units must be in celsius"
    assert location_rh_cube.units == '%', 'Ensure humidity units are in %'

    tas_cube_fahrenheit = location_tas_cube.copy()
    tas_cube_fahrenheit.convert_units('Fahrenheit') # Convert temperature cube to Fahrenheit
    
    tas_data = tas_cube_fahrenheit.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time
    humidity_data = location_rh_cube.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time

    heat_index_9_data = (
        16.923
        + 0.185212 * tas_data
        + 5.37941 * humidity_data
        - 0.100254 * tas_data * humidity_data
        + 9.4169e-3 * tas_data ** 2
        + 7.28898e-3 * humidity_data ** 2
        + 3.45372e-4 * tas_data ** 2 * humidity_data
        - 8.14971e-4 * tas_data * humidity_data ** 2
        + 1.02102e-5 * tas_data ** 2 * humidity_data ** 2
        - 3.8646e-5 * tas_data ** 3
        + 2.91583e-5 * humidity_data ** 3
        + 1.42721e-6 * tas_data ** 3 * humidity_data 
        + 1.97483e-7 * tas_data * humidity_data ** 3
        - 2.18429e-8 * tas_data ** 3 * humidity_data ** 2
        + 8.43296e-10 * tas_data ** 2 * humidity_data ** 3
        - 4.81975e-11 * tas_data ** 3 * humidity_data ** 3
        + 0.5
    )

    # Apply correction factor
    correction_mask = tas_data < 75
    heat_index_9_data[correction_mask] = tas_data[correction_mask]

    # Create a new cube with the heat index data
    heat_index_cube = location_tas_cube.copy()
    heat_index_cube.data = (heat_index_9_data - 32) * 5/9
    heat_index_cube.long_name = 'Heat Index 9'
    
    return heat_index_cube

# Function to calculate Heat Index (HI) using algorithm 10 above
def heat_index_10(location_tas_cube, location_rh_cube):
    """
    Returns the Blazejczyk et al. (2012) heat index from a tas and h cube.
    
    Parameters
    ----------
    location_tas_cube :
        An instance of `iris.cube.Cube` representing temperature data in ˚C
    location_rh_cube :
        An instance of `iris.cube.Cube` representing humidity data in %
    """
    assert location_tas_cube.units == 'celsius', "Temperature units must be in celsius"
    assert location_rh_cube.units == '%', 'Ensure humidity units are in %'

    tas_cube_copy = location_tas_cube.copy()  # Create a copy of the temperature cube

    tas_data = tas_cube_copy.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time
    humidity_data = location_rh_cube.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time

    heat_index_10_data = (
        -8.784695
        + 1.161139411 * tas_data
        + 2.338549 * humidity_data
        - 0.14611605 * tas_data * humidity_data
        - 1.2308094e-2 * tas_data ** 2
        - 1.6424828e-2 * humidity_data ** 2
        + 2.211732e-3 * tas_data ** 2 * humidity_data
        + 7.2546e-4 * tas_data * humidity_data ** 2
        - 3.582e-6 * tas_data ** 2 * humidity_data ** 2
    )

    # Apply correction factor
    correction_mask = tas_data <= 20
    heat_index_10_data[correction_mask] = tas_data[correction_mask]

    # Create a new cube with the heat index data
    heat_index_cube = location_tas_cube.copy()
    heat_index_cube.data = heat_index_10_data
    heat_index_cube.long_name = 'Heat Index 10'
    
    return heat_index_cube

# Function to calculate Heat Index (HI) using algorithm 11 above
def heat_index_11(location_tas_cube, location_rh_cube):
    """
    Returns the Patricola and Cook (2010) heat index from a tas and h cube.
    
    Parameters
    ----------
    location_tas_cube :
        An instance of `iris.cube.Cube` representing temperature data in ˚C
    location_rh_cube :
        An instance of `iris.cube.Cube` representing humidity data in %
    """
    assert location_tas_cube.units == 'celsius', "Temperature units must be in celsius"
    assert location_rh_cube.units == '%', 'Ensure humidity units are in %'

    tas_cube_copy = location_tas_cube.copy()  # Create a copy of the temperature cube
    tas_cube_copy.convert_units('Fahrenheit') # Convert temperature cube to Fahrenheit

    tas_data = tas_cube_copy.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time
    humidity_data = location_rh_cube.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time

    heat_index_11_data = (
        -42.4
        + 2.05 * tas_data
        + 10.1 * humidity_data
        - 0.255 * tas_data * humidity_data
        - (6.84e-3) * tas_data ** 2
        - (5.48e-2) * humidity_data ** 2
        + (1.23e-3) * tas_data ** 2 * humidity_data
        + (8.53e-4) * tas_data * humidity_data ** 2
        - (1.99e-6) * tas_data ** 2 * humidity_data ** 2
    )

    # Apply correction factor
    
    correction_mask = np.logical_or(tas_data <= 80, humidity_data <= 40)
    heat_index_11_data[correction_mask] = tas_data[correction_mask]

    # Create a new cube with the heat index data
    heat_index_cube = location_tas_cube.copy()
    heat_index_cube.data = (heat_index_11_data -32) * 5/9
    heat_index_cube.long_name = 'Heat Index 11'

    return heat_index_cube

# Function to calculate Heat Index (HI) using algorithm 12 above
def heat_index_12(location_tas_cube, location_td_cube):
    """
    Returns the Smoyer-Tomic and Rainham (2001) heat index from a tas and td cube.
    
    Parameters
    ----------
    location_tas_cube :
        An instance of `iris.cube.Cube` representing temperature data in ˚C
    location_td_cube :
        An instance of `iris.cube.Cube` representing dew point temperature data in ˚C
    """
    assert location_tas_cube.units == 'celsius' and location_td_cube.units == 'celsius', "Ensure cube units are in celsius"

    tas_cube = location_tas_cube.copy()
    tas_data = tas_cube.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time
    
    td_cube = location_td_cube.copy()
    dc_data = td_cube.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time

    heat_index_12_data = (
        -2.719
        + 0.994 * tas_data
        + 0.016 * dc_data ** 2
    )

    # Apply correction factor
    correction_mask = tas_data < 25
    heat_index_12_data[correction_mask] = tas_data[correction_mask]

    # Create a new cube with the heat index data
    heat_index_cube = location_tas_cube.copy()
    heat_index_cube.data = heat_index_12_data
    heat_index_cube.long_name = 'Heat Index 12'

    return heat_index_cube

# Function to calculate Heat Index (HI) using algorithm 13 above
def heat_index_13(location_tas_cube, location_td_cube):
    """
    Returns the O'Neill et al. (2003) heat index from a tas and td cube.
    
    Parameters
    ----------
    location_tas_cube :
        An instance of `iris.cube.Cube` representing temperature data in ˚C
    location_td_cube :
        An instance of `iris.cube.Cube` representing dew point temperature data in ˚C
    """
    assert location_tas_cube.units == 'celsius', "Temperature units must be in celsius"

    tas_cube_copy = location_tas_cube.copy()  # Create a copy of the temperature cube
    tas_data = tas_cube_copy.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time
    td_cube_copy = location_td_cube.copy()  # Create a copy of the temperature cube
    dc_data = td_cube_copy.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time

    heat_index_13_data = (
        -2.653
        + 0.994 * tas_data
        + 0.0153 * dc_data ** 2
    )

    # Create a new cube with the heat index data
    heat_index_cube = location_tas_cube.copy()
    heat_index_cube.data = heat_index_13_data
    heat_index_cube.long_name = 'Heat Index 13'

    return heat_index_cube

# Function to calculate Heat Index (HI) using algorithm 14 above
def heat_index_14(location_tas_cube, location_td_cube):
    """Returns the Perry et al. (2011) heat index from a tas and td cube.
    
    Parameters
    ----------
    location_tas_cube :
        An instance of :class:`iris.cube.Cube`
    location_td_cube :
        An instance of :class:`iris.cube.Cube`
    """
    assert location_tas_cube.units == 'celsius' and location_td_cube.units == 'celsius', "Ensure cube units are in celsius"

    tas_cube_copy = location_tas_cube.copy()  # Create a copy of the temperature cube
    td_cube_copy = location_td_cube.copy()  # Create a copy of the td cube

    heat_index_14_data = (-2.719 
                          + 0.994 * tas_cube_copy.data 
                          + 0.016 * td_cube_copy.data ** 2)

    heat_index_cube = tas_cube_copy.copy()  # Create a new cube with the heat index data
    heat_index_cube.data = heat_index_14_data
    heat_index_cube.long_name = 'Heat Index 14'

    return heat_index_cube

# Function to calculate Heat Index (HI) using algorithm 15 above
def heat_index_15(location_tas_cube, location_rh_cube):
    """
    Returns the Tam et al. (2008) heat index from a tas and h cube.
    
    Parameters
    ----------
    location_tas_cube :
        An instance of `iris.cube.Cube` representing temperature data in ˚C
    location_rh_cube :
        An instance of `iris.cube.Cube` representing humidity data in %
    """
    assert location_tas_cube.units == 'celsius', "Temperature units must be in celsius"
    assert location_rh_cube.units == '%', 'Ensure humidity units are in %'

    tas_cube_fahrenheit = location_tas_cube.copy()
    tas_cube_fahrenheit.convert_units('Fahrenheit') # Convert temperature cube to Fahrenheit
    
    tas_data = tas_cube_fahrenheit.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time
    humidity_data = location_rh_cube.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time

    heat_index_15_data = (
        -42.379
        + 2.049015 * tas_data
        + 10.1433 * humidity_data
        - 0.2248 * tas_data * humidity_data
        - (6.83783e-3) * tas_data ** 2
        - (5.4817e-2) * humidity_data ** 2
        + (1.229e-3) * tas_data ** 2 * humidity_data
        + (8.528e-4) * tas_data * humidity_data ** 2
        - (1.99e-6) * tas_data ** 2 * humidity_data ** 2
    )

    # Apply correction factor
    correction_mask = tas_data < 57
    heat_index_15_data[correction_mask] = tas_data[correction_mask]

    # Create a new cube with the heat index data
    heat_index_cube = location_tas_cube.copy()
    heat_index_cube.data = (heat_index_15_data -32) * 5/9
    heat_index_cube.long_name = 'Heat Index 15'

    return heat_index_cube

# Function to calculate Heat Index (HI) using algorithm 16 above
def heat_index_16(location_tas_cube, location_rh_cube):
    """
    Returns the Rothfusz (1990) heat index from a tas and h cube.
    
    Parameters
    ----------
    location_tas_cube :
        An instance of `iris.cube.Cube` representing temperature data in ˚C
    location_rh_cube :
        An instance of `iris.cube.Cube` representing humidity data in %
    """
    assert location_tas_cube.units == 'celsius', "Temperature units must be in celsius"
    assert location_rh_cube.units == '%', 'Ensure humidity units are in %'

    tas_cube_fahrenheit = location_tas_cube.copy()
    tas_cube_fahrenheit.convert_units('Fahrenheit') # Convert temperature cube to Fahrenheit
    
    tas_data = tas_cube_fahrenheit.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time
    humidity_data = location_rh_cube.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time

    heat_index_16_data = (
        -42.379
        + 2.04901523 * tas_data
        + 10.14333127 * humidity_data
        - 0.22475541 * tas_data * humidity_data
        - (6.83783e-3) * tas_data ** 2
        - (5.481717e-2) * humidity_data ** 2
        + (1.22874e-3) * tas_data ** 2 * humidity_data
        + (8.5282e-4) * tas_data * humidity_data ** 2
        - (1.99e-6) * tas_data ** 2 * humidity_data ** 2
    )

    # Create a new cube with the heat index data
    heat_index_cube = location_tas_cube.copy()
    heat_index_cube.data = (heat_index_16_data -32) * 5/9
    heat_index_cube.long_name = 'Heat Index 16'

    return heat_index_cube

# Function to calculate Heat Index (HI) using algorithm 17 above
def heat_index_17(location_tas_cube, location_rh_cube):
    """
    Returns the Fischer and Schär (2010) heat index from a tas and h cube.
    
    Parameters
    ----------
    location_tas_cube :
        An instance of `iris.cube.Cube` representing temperature data in ˚C
    location_rh_cube :
        An instance of `iris.cube.Cube` representing humidity data in %
    """
    assert location_tas_cube.units == 'celsius', "Temperature units must be in celsius"
    assert location_rh_cube.units == '%', 'Ensure humidity units are in %'

    tas_cube_copy = location_tas_cube.copy()  # Create a copy of the temperature cube

    tas_data = tas_cube_copy.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time
    humidity_data = location_rh_cube.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time

    heat_index_17_data = (
        -8.7847 
        + 1.6114 * tas_data 
        - 0.012308 * tas_data ** 2
        + (humidity_data * (2.3385 - 0.14612 * tas_data 
                           + 2.2117e-3 * tas_data ** 2))
        + (humidity_data ** 2 * (-0.016425 + 7.2546e-4 * tas_data))
        + (-3.582e-6) * tas_data ** 2)
    
    # Create a new cube with the heat index data
    heat_index_cube = location_tas_cube.copy()
    heat_index_cube.data = heat_index_17_data
    heat_index_cube.long_name = 'Heat Index 17'

    return heat_index_cube

# Function to calculate Heat Index (HI) using algorithm 18 above
def heat_index_18(location_tas_cube, location_rh_cube):
    """
    Returns the Costanzo et al. (2006) heat index from a tas and h cube.
    
    Parameters
    ----------
    location_tas_cube :
        An instance of `iris.cube.Cube` representing temperature data in ˚C
    location_rh_cube :
        An instance of `iris.cube.Cube` representing humidity data in %
    """
    assert location_tas_cube.units == 'celsius', "Temperature units must be in celsius"
    assert location_rh_cube.units == '%', 'Ensure humidity units are in %'

    tas_cube_copy = location_tas_cube.copy()  # Create a copy of the temperature cube

    tas_data = tas_cube_copy.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time
    humidity_data = location_rh_cube.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time

    heat_index_18_data = (tas_data 
                          - 0.55 * (1 - 0.001 * humidity_data) 
                          * (tas_data - 14.5))
    
    # Create a new cube with the heat index data
    heat_index_cube = location_tas_cube.copy()
    heat_index_cube.data = heat_index_18_data
    heat_index_cube.long_name = 'Heat Index 18'

    return heat_index_cube

# Function to calculate Heat Index (HI) using algorithm 19 above
def heat_index_19(location_tas_cube, location_td_cube):
    """
    Returns the Smoyer (1998) heat index from a tas and td cube.
    
    Parameters
    ----------
    location_tas_cube :
        An instance of `iris.cube.Cube` representing temperature data in ˚C
    location_td_cube :
        An instance of `iris.cube.Cube` representing dew point temperature data in ˚C
    """
    assert location_tas_cube.units == 'celsius' and location_td_cube.units == 'celsius', "Ensure cube units are in celsius"
    
    tas_cube_copy = location_tas_cube.copy()  # Create a copy of the temperature cube
    tas_data = tas_cube_copy.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time
    td_cube_copy = location_td_cube.copy()  # Create a copy of the td cube
    td_data = td_cube_copy.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time

    heat_index_19_data = (2.719 
                          + (0.994 * tas_data) 
                          + (0.016 * (td_data ** 2)))
    
    # Create a new cube with the heat index data
    heat_index_cube = location_tas_cube.copy()
    heat_index_cube.data = heat_index_19_data
    heat_index_cube.long_name = 'Heat Index 19'

    return heat_index_cube

# Function to calculate Heat Index (HI) using algorithm 20 above
def heat_index_20(location_tas_cube, location_rh_cube):
    """
    Returns the Lajinian et al. (1997) heat index from a tas and h cube.
    
    Parameters
    ----------
    location_tas_cube :
        An instance of `iris.cube.Cube` representing temperature data in ˚C
    location_rh_cube :
        An instance of `iris.cube.Cube` representing humidity data in %
    """
    assert location_tas_cube.units == 'celsius', "Temperature units must be in celsius"
    assert location_rh_cube.units == '%', 'Ensure humidity units are in %'

    tas_cube_fahrenheit = location_tas_cube.copy()
    tas_cube_fahrenheit.convert_units('Fahrenheit') # Convert temperature cube to Fahrenheit
    
    tas_data = tas_cube_fahrenheit.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time
    humidity_data = location_rh_cube.data.astype(np.float64)  # Convert the cube data to NumPy arrays to reduce processing time

    heat_index_20_data = (tas_data
                          - ((0.55 - 0.55 * (humidity_data / 100)) 
                             * tas_data - 58)
    )

    # Create a new cube with the heat index data
    heat_index_cube = location_tas_cube.copy()
    heat_index_cube.data = (heat_index_20_data -32) * 5/9
    heat_index_cube.long_name = 'Heat Index 20'

    return heat_index_cube

# Function to calculate Heat Index (HI) using algorithm 21 above
def heat_index_21(location_tas_cube, location_td_cube):
    """
    Returns the Basara et al. (2010) heat index from a tas and td cube.
    
    Parameters
    ----------
    location_tas_cube :
        An instance of `iris.cube.Cube` representing temperature data in ˚C
    location_td_cube :
        An instance of `iris.cube.Cube` representing dew point temperature data in ˚C
    """

    tas_cube = location_tas_cube.copy()  # Create a copy of the temperature cube
    td_cube = location_td_cube.copy()  # Create a copy of the td cube
    
    heat_index_21_data = (
        -2.653 
        + 0.994 * tas_cube.data
        + 0.368 * (td_cube.data ** 2)
    )

    # Create a new cube with the heat index data
    heat_index_cube = location_tas_cube.copy()
    heat_index_cube.data = heat_index_21_data
    heat_index_cube.long_name = 'Heat Index 21'

    return heat_index_cube

In [ ]:
# Creating all the heat indices cubes in one go
def create_all_21_heat_indices_cubes(tas_cube, rh_cube, td_cube, es_cube):
    global heat_indices_cubelist
    """
    Returns heat index values from a tas and h cube for all 21 algorithms from Anderson et al. (2013).

    Parameters
    ----------
    location_tas_cube :
        An instance of `iris.cube.Cube` representing temperature data in ˚C
    location_rh_cube :
        An instance of `iris.cube.Cube` representing humidity data in %
    """
    tas_cube.convert_units('celsius')
    td_cube.convert_units('celsius')

    output_folder = 'heat_index_cubes'
    if not os.path.exists(output_folder):
        os.mkdir(output_folder)
    
    with open('basic_info.py', 'a') as f:
            f.write(f"\nheat_index_cubes = '{output_folder}'")
    
    heat_index_functions = [
        heat_index_1, heat_index_2, heat_index_3, heat_index_4, heat_index_5,
        heat_index_6, heat_index_7, heat_index_8, heat_index_9, heat_index_10,
        heat_index_11, heat_index_12, heat_index_13, heat_index_14, heat_index_15,
        heat_index_16, heat_index_17, heat_index_18, heat_index_19, heat_index_20,
        heat_index_21
    ]
    
    for index, heat_index_function in enumerate(heat_index_functions):
        filename = f'{output_folder}/heat_index_{index+1}_cube.nc'
        if not os.path.exists(filename):
            print(f'Creating heat index {index+1} cube...')
            if index in [0, 4, 5, 6, 7, 8, 9, 10, 14, 15, 16, 17, 19]:
                heat_index_cube = heat_index_function(tas_cube, rh_cube)
            elif index in [1, 2, 11, 12, 13, 18, 20]:
                heat_index_cube = heat_index_function(tas_cube, td_cube)
            elif index == 3:
                heat_index_cube = heat_index_function(tas_cube, es_cube)
            else:
                print(f'Index {index} does not match any condition. Skipping creation')
                continue

            print(f'Heat index {index+1} cube created, saving as {filename}')
            save_cube(heat_index_cube, filename)
        else:
            print(f'Heat index {index+1} cube already exists, skipping.')

In [ ]:
create_all_21_heat_indices_cubes(tas_cube, rh_cube, td_cube, es_cube)